In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.sparse as sps
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [9., 6.]

# Building a parametric `LTIModel`

In [ ]:
n = 100

E = sps.eye(n, format='lil')
E[0, 0] = E[-1, -1] = 0.5
E = E.tocsc()

alpha = 1.172e-5
N = (n - 1)**2
A0 = sps.diags([(n - 1) * [N], n * [-2 * N], (n - 1) * [N]], [-1, 0, 1], format='lil')
A0[0, 0] = -(n - 1) * n
A0[-1, -1] = -(n - 1) * n
A1 = sps.lil_matrix((n, n))
A1[:n//2, :] = alpha * A0[:n//2, :]
A1 = A1.tocsc()
A2 = sps.lil_matrix((n, n))
A2[n//2:, :] = A0[n//2:, :]
A2 = A2.tocsc()

B = np.zeros((n, 1))
B[0, 0] = alpha * (n - 1)

C = np.zeros((2, n))
C[0, -1] = 1
C[1, :] = 1/n

In [ ]:
from pymor.models.iosys import LTIModel
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.parameters.functionals import ProjectionParameterFunctional

In [ ]:
Eop = NumpyMatrixOperator(E)

A1op = NumpyMatrixOperator(A1)
A2op = NumpyMatrixOperator(A2)
Aop = A1op + ProjectionParameterFunctional('p') * A2op

Bop = NumpyMatrixOperator(B)

Cop = NumpyMatrixOperator(C)

In [ ]:
fom = LTIModel(Aop, Bop, Cop, E=Eop)

In [ ]:
fom

In [ ]:
print(fom)

In [ ]:
fom.parameters

## Poles

Compute and plot the poles for $p \in \{10^{-6}, 10^{-3}, 1\}$.

## Magnitude plot

Plot the magnitude plot for $p \in \{10^{-6}, 10^{-3}, 1\}$ and frequencies in the interval $[10^{-8}, 10]$.

# Hankel singular values

Plot the hankel singular values for $p \in \{10^{-6}, 10^{-3}, 1\}$.

## $\mathcal{H}_2$ norms

Compute and plot the $\mathcal{H}_2$ norm of $10$ different models for $p \in [10^{-6}, 1]$.

## $\mathcal{H}_\infty$ norms

Compute and plot the $\mathcal{H}_\infty$ norm of $10$ different models for $p \in [10^{-6}, 1]$.

# Balanced Truncation

In the following our goal is to compute a reduced model which approximates the input-output behaviour of the full order for $p \in \{10^{-6}, 10^{-3}, 1\}$. In order to do so, we first use the `BTReductor` to compute projection matrices with respect to each individual parameter. The final projection matrix is then constructed via pyMORs `cat_arrays` and `qr_svd` functions. 

In [ ]:
from pymor.reductors.bt import BTReductor
from pymor.vectorarrays.constructions import cat_arrays

In [ ]:
p_list = [1e-6, 1e-3, 1]
V = []
W = []
for p in p_list:
    bt = BTReductor(fom, mu=p)
    rom = bt.reduce(10)
    V.append(bt.V)
    W.append(bt.W)
V = cat_arrays(V)
W = cat_arrays(W)

In [ ]:
from pymor.algorithms.svd_va import qr_svd

In [ ]:
V2, s, Vh = qr_svd(cat_arrays([V, W]))

In [ ]:
_ = plt.semilogy(s, '.-')

Use the `LTIPGReductor` to project the model via the computed bases.

In [ ]:
from pymor.reductors.basic import LTIPGReductor

In [ ]:
pg = LTIPGReductor(...)
rom = ...

In [ ]:
err = fom - rom

## Poles

Compute and plot the poles of the reduced order model for $p \in \{10^{-6}, 10^{-3}, 1\}$.

## Relative $\mathcal{H}_2$ errors

Compute and plot the relative $\mathcal{H}_2$ error of $10$ different reduced models for $p \in [10^{-6}, 1]$.

## Relative $\mathcal{H}_\infty$ errors

Compute and plot the relative $\mathcal{H}_\infty$ error of $10$ different reduced models for $p \in [10^{-6}, 1]$.